In [205]:
import sys
import os
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
params = {'text.usetex' : False,
          'font.size' : 36,
          'legend.fancybox':True,
          'legend.loc' : 'best',

          'legend.framealpha': 0.9,
          "legend.fontsize" : 21,
         }
plt.rcParams.update(params)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
if os.path.exists('./core'):
    os.remove('./core')

In [206]:
try:
    import google.colab
    IN_COLAB = True
    !pip install git+https://github.com/AndreiChertkov/teneva.git
    !pip install tntorch
except:
    IN_COLAB = False

  Cloning https://github.com/AndreiChertkov/teneva.git to /tmp/pip-req-build-9cxg68w0
  Running command git clone --filter=blob:none --quiet https://github.com/AndreiChertkov/teneva.git /tmp/pip-req-build-9cxg68w0
  Resolved https://github.com/AndreiChertkov/teneva.git to commit 953ca05731826232d0bee34030ff6f78e0a099c6
  Preparing metadata (setup.py) ... done


In [207]:
d    = 2                          # Dimension of the function
a    = [0,0]  # Lower bounds for spatial grid
b    = [5,5,]  # Upper bounds for spatial grid
n    = [1024,1024]  # Shape of the tensor
m    = 5.E+6                     # Number of calls to target function
nswp = 10                         # Sweep number for ALS iterations
r    = 20

In [208]:
import teneva
import tntorch
from teneva import cross
from time import perf_counter as tpc
from math import sin , cos , atan2, pi , atan
import tntorch as tn

In [209]:
def set_seed(seed=4242):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

In [210]:
set_seed(7)

In [211]:
def func(I,
         w2=10,
         parameter_static_X = [0,2],
         parameter_static_Y = [0,2],is_teneva=False,is_index=False):
    if is_index or is_teneva:
      X = teneva.ind_to_poi(I, a, b, n)
    else:
      # is tntorch or its cheb grid aka func-tt
      X = I[:]
    j2 = []
    #print(X)
    for parameter_static_x,parameter_static_y in zip(parameter_static_X,parameter_static_Y):
        j = []
        for x,y in X:
            distance = (parameter_static_x-x)**2/ 0.5**2 + (parameter_static_y-y)**2 / 0.5**2
            obst_stat = 5*(pi/2 + atan(w2 - distance*w2))
            j.append(obst_stat)
        j2.append(j)
    j2 = np.array(j2).T
    j = []
    assert len(j2) == len(I)
    n_vars = len(parameter_static_X)
    for k in range(len(I)):
        m = np.random.choice(np.arange(n_vars))
        #m = k % n_vars
        j.append(j2[k,m])
    j = np.array(j)
    if not is_teneva:
      j= torch.from_numpy(j)
    return j

In [212]:
def _get_many(Y,I):
  return np.array([float(Y[i[0],i[1]]) for i in I])

In [213]:
def make_plot(Y,I_trn,I_tst,is_teneva=True,is_div=False):
  X_trn = teneva.ind_to_poi(I_trn, a, b, n)
  X_tst = teneva.ind_to_poi(I_tst, a, b, n)
  if is_teneva:
    y_trn = teneva.get_many(Y, I_trn)
    y_tst = teneva.get_many(Y, I_tst)
  else:
    y_trn = _get_many(Y, I_trn)
    y_tst = _get_many(Y, I_tst)
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(111, projection='3d')
  ### train plot
  arr = []
  for x,y in zip(X_trn,y_trn):
      arr.append([y,x[0],x[1]])
  arr = np.array(arr)
  th = 0.5
  indeces = np.where(arr[:,0] < th)[0]
  indeces = np.random.choice(indeces, round(len(indeces)*0.005))
  indeces2 = np.where(arr[:,0] >= th)[0]
  arr = np.concatenate ((arr[indeces], arr[indeces2]))
  if not is_div:
    ax.scatter(arr[:,1], arr[:,2], arr[:,0],color='green',label='j2',alpha=0.1)
  arr = []
  for x,y in zip(X_trn,y_our_train):
      arr.append([y,x[0],x[1]])
  arr = np.array(arr)
  indeces = np.where(arr[:,0] < th)[0]
  indeces = np.random.choice(indeces, round(len(indeces)*0.005))
  indeces2 = np.where(arr[:,0] >= th)[0]
  arr = np.concatenate ((arr[indeces], arr[indeces2]))

  ax.scatter(arr[:,1], arr[:,2], arr[:,0],color='red',label='tt-cross',alpha=0.1)

  ##### test plot
  arr=[]
  for x,y in zip(X_tst,y_tst):
      arr.append([y,x[0],x[1]])
  arr = np.array(arr)
  th = 0.5
  indeces = np.where(arr[:,0] < th)[0]
  indeces = np.random.choice(indeces, round(len(indeces)*0.005))
  indeces2 = np.where(arr[:,0] >= th)[0]
  arr = np.concatenate ((arr[indeces], arr[indeces2]))

  ax.scatter(arr[:,1], arr[:,2],arr[:,0],color='red',label='tt-cross (test)',marker='x')

  arr=[]
  for x,y in zip(X_tst,y_our_test):
      arr.append([y,x[0],x[1]])
  arr = np.array(arr)
  th = 0.5
  indeces = np.where(arr[:,0] < th)[0]
  indeces = np.random.choice(indeces, round(len(indeces)*0.005))
  indeces2 = np.where(arr[:,0] >= th)[0]
  arr = np.concatenate ((arr[indeces], arr[indeces2]))
  if not is_div:
    ax.scatter(arr[:,1], arr[:,2],
            arr[:,0],color='green',label='j2',marker='x')

  plt.legend()
  plt.tight_layout()
  plt.show()

## Cross

In [214]:

max_iter = nswp*d*n[0]*r*r
I_trn = teneva.sample_lhs(n, m)
y_trn = func(I_trn,is_teneva=True)
m_tst = int(1.E+4)

# Random multi-indices for the test points:
I_tst = np.vstack([np.random.choice(k, m_tst) for k in n]).T

# Function values for the test points:
y_tst = func(I_tst,is_teneva=True)

In [215]:
assert set(tuple(e) for e in I_tst) not in set(tuple(e) for e in I_trn)

In [216]:
cheb = lambda I: func(teneva.ind_to_poi(I, a, b, n, 'cheb'),is_teneva=False)

In [ ]:
e = None
dr_min    = 1   # Cross parameter (minimum number of added rows)
dr_max    = 7      # Cross parameter (maximum number of added rows)

t = tpc()
info, cache = {}, {}
#Y = teneva.anova(I_trn, y_trn, r)
#Y = teneva.als(I_trn, y_trn, Y, nswp)
Y = teneva.rand(n, r)
Y = cross(cheb, Y, m, e, nswp, dr_min=dr_min, dr_max=dr_max,
    info=info, cache=cache)
Y = teneva.truncate(Y, 1.E-4) # We round the result at the end
t = tpc() - t

print(f'Build time           : {t:-10.2f}')
print(f'Evals func           : {info["m"]:-10d}')
print(f'Cache uses           : {info["m_cache"]:-10d}')
print(f'Iter accuracy        : {info["e"]:-10.2e}')
print(f'Sweep number         : {info["nswp"]:-10d}')
print(f'Stop condition       : {info["stop"]:>10}')
print(f'TT-rank of pure res  : {info["r"]:-10.1f}')
print(f'TT-rank of trunc res : {teneva.erank(Y):-10.1f}')

# Compute approximation in train points:
y_our_train = teneva.get_many(Y, I_trn)

# Accuracy of the result for train points:
e_trn = np.linalg.norm(y_our_train - y_trn)
e_trn /= np.linalg.norm(y_trn)

# Compute approximation in test points:
y_our_test = teneva.get_many(Y, I_tst)

# Accuracy of the result for test points:
e_tst = np.linalg.norm(y_our_test - y_tst)
e_tst /= np.linalg.norm(y_tst)

print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')

In [ ]:
X_trn = teneva.ind_to_poi(I_trn, a, b, n)
X_tst = teneva.ind_to_poi(I_tst, a, b, n)
A = teneva.func_int(Y)

In [ ]:
teneva.func_get(X_trn[:1], A, a, b, z=-1.),func(X_trn[:1])

In [ ]:
make_plot(Y,I_trn,I_tst)

In [ ]:
6

In [ ]:
assert

# ALS

In [ ]:


t = tpc()
nswp = 10
Y_als = teneva.anova(I_trn, y_trn, r)
Y_als = teneva.als(I_trn, y_trn, Y, nswp)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')

In [ ]:


# Compute approximation in train points:
y_our_train = teneva.get_many(Y, I_trn)

# Accuracy of the result for train points:
e_trn = np.linalg.norm(y_our_train - y_trn)
e_trn /= np.linalg.norm(y_trn)

# Compute approximation in test points:
y_our_test = teneva.get_many(Y, I_tst)

# Accuracy of the result for test points:
e_tst = np.linalg.norm(y_our_test - y_tst)
e_tst /= np.linalg.norm(y_tst)

print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')



# Tntorch

In [ ]:


#t = tpc()
domain = [torch.linspace(a[0],b[0],n[0]) for i in range(d)] ## is this correct?
#domain = [torch.arange(a[0],b[0]) for i in range(d)]  # works but how to make fine grid?
Ytn,info_ = tn.cross(function=func,function_arg='matrix',return_info=True,
                  rmax=info['r'],tensors=tn.meshgrid(domain),max_iter=10)

#t2 = tn.cross(function=lambda x: x, tensors=Ytn)


t = tpc() - t




In [ ]:
Y_init = tn.randn(64, 64, ranks_tt=24, requires_grad=True)
Y_init.cores = [torch.tensor(y,requires_grad=True) for y in Y_als]

In [ ]:
Y_fwd,info_ = tn.cross(function=lambda x: x, tensors=Y_init,return_info=True)
Y_fwd = tn.cross_forward(info_,tensors=Y_init)
#Y_fwd,info_ = tn.cross(function=func,function_arg='matrix',return_info=True,
#                 rmax=info['r'],tensors=Y_init,max_iter=10)

In [ ]:
Y_fwd

In [ ]:
make_plot(Y_init,I_trn,I_tst,is_teneva=False)

In [ ]:
make_plot(Y_fwd,I_trn,I_tst,is_teneva=False)

In [ ]:
x = Y_fwd.torch()[0,0]

In [ ]:
x.grad

In [ ]:
make_plot(dY,I_trn,I_tst,is_teneva=False,is_div=True)

In [ ]:
dY

Simple Test for grad

In [ ]:
domain

In [ ]:
t[0,0]

In [ ]:
t[0,0],t[63,63]